In this demo we'll use two nifty but lesser known Python libraries ([OSMnx](https://github.com/gboeing/osmnx) & [Folium](https://github.com/python-visualization/folium)) to make an agent that helps find travel routes and plots them.

This is a more difficult task than some of our other examples, so we'll make use of GPT5.

In [ ]:
from agex import Agent, Versioned, connect_llm, MemberSpec
from agex.helpers import register_stdlib
import osmnx as ox
import networkx as nx
import folium
import geopandas as gpd

routy = Agent(
    name="routy",
    primer="You assist the user finding routes (OSMnx) and visualizing them (Folium).",
    llm_client=connect_llm(provider="openai", model="gpt-5"),
    timeout_seconds=60.0,  # agent will need to download openstreetmap data
)

Now we'll start registration with our agent.

This is an alternative to tool-layer abstractions. Instead we're unlocking Python abilities for our agent and highlighting useful bits as guidance.

In [ ]:
# give our agent standard lib modules like 'math'
register_stdlib(routy)

# our agent may not be familiar with osmnx, so we'll highlight parts
routy.module(
    ox,
    visibility="low",
    recursive=True,
    configure={
        # elevate for high visibility
        "geocoder.geocode": MemberSpec(visibility="high"),
        "geocoder.geocode_to_gdf": MemberSpec(visibility="high"),
        "graph.graph_from_point": MemberSpec(visibility="high"),
        "graph.graph_from_bbox": MemberSpec(visibility="high"),
        "distance.nearest_nodes": MemberSpec(visibility="high"),
        "distance.nearest_edges": MemberSpec(visibility="high"),
        "distance.great_circle": MemberSpec(visibility="high"),
        "routing.shortest_path": MemberSpec(visibility="high"),
        "routing.k_shortest_paths": MemberSpec(visibility="high"),
        "routing.route_to_gdf": MemberSpec(visibility="high"),
        "convert.graph_to_gdfs": MemberSpec(visibility="high"),
    },
)

# our agent may not be familiar with folium, so we'll highlight parts
routy.module(
    folium,
    visibility="low",
    recursive=True,
    configure={
        # elevate for high visibility
        "Map": MemberSpec(visibility="high"),
        "Marker": MemberSpec(visibility="high"),
        "PolyLine": MemberSpec(visibility="high"),
    },
)

# as osmnx uses nx and geopandas artifacts, our agent may want these as well
routy.module(nx, visibility="low", recursive=True)
routy.module(gpd, visibility="low", recursive=True)

Next we make the agent entry point.

We define it just like a regular Python fn. The types are the contracts and the agent gets to build the implementation dynamically when the fn is called.

In [3]:
@routy.task
def route(prompt: str) -> folium.Map:  # type: ignore[return-value]
    "Find a route given the prompt and return a Folium map."
    pass

Let's call this new task function.

First we'll create a `Versioned` state so that our agent has memory accross task calls.

In [4]:
state = Versioned()
map = route("I'd like drive from Albany, OR to Corvallis, OR", state=state)
display(map)

Awesome! The agent found a straight forward route and gave us an interactive `folium.Map`.

But oh no, it turns out US20 is closed due to [slime eels](https://www.youtube.com/watch?v=WZaFlydVHAU&ab_channel=KGWNews)!

In [ ]:
map = route("Hwy 20 is closed between Albany and Corvallis, other options?", state=state)
display(map)

Great, the agent found two viable alternatives!

Under the hood, the agent is downloading openstreetmap data, geocoding locations, searching for viable routes, and then converting routes into plottable polylines. Its a considerable amount of work.

Excitingly, agents often end up creating their own helper fns. When using stateful task calls, these helpers survive and an agent may reuse them (building their own toolset!).

```python
# Helper: compute shortest path while avoiding edges with certain patterns in name
def compute_route_avoiding(G_in, orig, dest, avoid_patterns_upper):
    H = G_in.copy()
    to_remove = []
    for u, v, k, data in H.edges(keys=True, data=True):
        ref = str(data.get('ref', '') or '')
        name = str(data.get('name', '') or '')
        tags = (ref + ' ' + name).upper()
        if any(p in tags for p in avoid_patterns_upper):
            to_remove.append((u, v, k))
    if to_remove:
        H.remove_edges_from(to_remove)
    route = osmnx.routing.shortest_path(H, orig=orig, dest=dest, weight='length')
    return H, route, len(to_remove)

# Alternative 1: Avoid US-20
patterns_us20 = ['US 20', 'US-20', 'US20', 'HWY 20', 'OR 20', 'OR-20', 'OR20']
H1, route1, removed1 = compute_route_avoiding(G, orig_node, dest_node, [p.upper() for p in patterns_us20])
...
```

All this possible without the hassle of pre-defining tooling layer on top of OSMnx or Folium. Instead, the agent stitches them together using the guidance already available within the libraries' documentation.